In [ ]:
# I want to figure out how to score promoter AI variants. It's relatively straightforward since we have a TSV file

In [13]:
import numpy as np
import pandas as pd
import pickle
import math

qtls = pd.read_csv('/data1/lesliec/sarthak/data/joint_playground/eQTL/EPCOTv2_LCLs/LCLs.txt', sep=' ', header=None)
qtls.columns = ['label', 'qtl_idx', 'gene_idx', 'chrom', 'gene_start', 'gene_end', 'strand', 'qtl_loc', 'ref', 'alt', 'sign_target']
#some other things to load in
tempgenome = np.load('/data1/lesliec/sarthak/data/chrombpnet_test/hg38_tokenized.npz')
genome_loaded = {}
for key in tempgenome:
    genome_loaded[key] = tempgenome[key][:]




base_dir = '/data1/lesliec/sarthak/data/joint_playground/eQTL/EPCOTv2_LCLs/'
with open(base_dir+'genes.pickle', 'rb') as f:
    gene_annotation = pickle.load(f)

ordered_genes = sorted(list(gene_annotation.keys()))

tmpgeneTSS = np.loadtxt(base_dir+'ensemblTSS.txt', dtype='str')
geneTSS_dic = {tmpgeneTSS[i, 0]: int(tmpgeneTSS[i, 1]) for i in range(tmpgeneTSS.shape[0])}

mapping = {'A':(1,0,0,0),'C':(0,1,0,0),'G':(0,0,1,0),'T':(0,0,0,1),'N':(0,0,0,0)}
mapping_nuc = {'A':7, 'C':8,'G':9,'T':10, 'N':11} #to help us determine what our nucleotide is

In [14]:
qtls

,label,qtl_idx,gene_idx,chrom,gene_start,gene_end,strand,qtl_loc,ref,alt,sign_target
0,0,0,10057,14,23953734,23969279,1,23953727,C,G,-1
1,0,0,10057,14,23953734,23969279,1,23953798,G,A,-1
2,0,0,10057,14,23953734,23969279,1,23954373,A,G,-1
3,0,0,10119,1,2555639,2565382,1,2557169,T,C,1
4,0,0,10119,1,2555639,2565382,1,2568341,G,A,1
...,...,...,...,...,...,...,...,...,...,...,...
1326,1,0,9772,3,12484421,12541549,1,12484507,G,A,1
1327,1,0,9844,14,22773222,22829820,-1,22803905,T,C,1
1328,1,0,9854,9,71911510,71986054,1,71941892,A,G,1
1329,1,0,9921,1,111117163,111140203,-1,111139496,C,T,-1


In [15]:
#let's also filter qtls to ones where the distance is less than 10k
qtls['distance'] = qtls['gene_start'] - qtls['qtl_loc']
qtls = qtls[(qtls['distance'] > -10000) & (qtls['distance'] < 10000)]
qtls

,label,qtl_idx,gene_idx,chrom,gene_start,gene_end,strand,qtl_loc,ref,alt,sign_target,distance
0,0,0,10057,14,23953734,23969279,1,23953727,C,G,-1,7
1,0,0,10057,14,23953734,23969279,1,23953798,G,A,-1,-64
2,0,0,10057,14,23953734,23969279,1,23954373,A,G,-1,-639
3,0,0,10119,1,2555639,2565382,1,2557169,T,C,1,-1530
5,0,0,10162,18,21650901,21704780,-1,21655045,A,G,1,-4144
...,...,...,...,...,...,...,...,...,...,...,...,...
1316,1,0,8681,4,186191567,186213463,1,186191475,G,C,-1,92
1317,1,0,873,11,2301997,2318204,1,2301990,G,A,-1,7
1319,1,0,8995,8,143574785,143578649,-1,143578375,A,G,-1,-3590
1320,1,0,9109,10,132332154,132381508,1,132331810,A,G,1,344


In [19]:
#let's sort it by first the chrom and then the qtlloc collumns
qtls = qtls.sort_values(by=['chrom', 'qtl_loc'])
qtls

,label,qtl_idx,gene_idx,chrom,gene_start,gene_end,strand,qtl_loc,ref,alt,sign_target,distance
1138,1,0,13893,1,1373730,1375207,-1,1374801,A,G,1,-1071
1169,1,0,17446,1,1512162,1534685,1,1511945,G,A,-1,217
554,0,0,34199,1,2549920,2557031,-1,2548544,A,G,1,1376
3,0,0,10119,1,2555639,2565382,1,2557169,T,C,1,-1530
1188,1,0,25588,1,6234692,6239444,1,6236178,T,G,-1,-1486
...,...,...,...,...,...,...,...,...,...,...,...,...
1173,1,0,1988,22,19479457,19520612,1,19480129,T,C,1,-672
864,0,0,5977,22,20080221,20111877,1,20080980,G,T,1,-759
1248,1,0,45491,22,46761893,46762564,-1,46762547,T,C,-1,-654
1286,1,0,60754,23,1661795,1662306,1,1662011,A,G,-1,-216


In [5]:
#now let's load in the large promoter ai tsv
promoterai = pd.read_csv('/data1/lesliec/sarthak/data/promoterAI/promoterAI_tss500.tsv', sep='\t', nrows=500)
promoterai

,chrom,pos,ref,alt,gene,gene_id,transcript_id,strand,tss_pos,promoterAI
0,chr1,64918,T,A,OR4F5,ENSG00000186092,ENST00000641515.2,1,65418,-0.0024
1,chr1,64918,T,C,OR4F5,ENSG00000186092,ENST00000641515.2,1,65418,-0.0076
2,chr1,64918,T,G,OR4F5,ENSG00000186092,ENST00000641515.2,1,65418,-0.0152
3,chr1,64919,A,C,OR4F5,ENSG00000186092,ENST00000641515.2,1,65418,-0.0027
4,chr1,64919,A,G,OR4F5,ENSG00000186092,ENST00000641515.2,1,65418,-0.0066
...,...,...,...,...,...,...,...,...,...,...
495,chr1,65083,T,A,OR4F5,ENSG00000186092,ENST00000641515.2,1,65418,0.0005
496,chr1,65083,T,C,OR4F5,ENSG00000186092,ENST00000641515.2,1,65418,0.0106
497,chr1,65083,T,G,OR4F5,ENSG00000186092,ENST00000641515.2,1,65418,0.0033
498,chr1,65084,T,A,OR4F5,ENSG00000186092,ENST00000641515.2,1,65418,-0.0010


In [28]:
#we already know that the position of ref and alternate of qtls is pos -1 for proper indexing, let's ensure with promoterai
#first redo for qtls

genome = np.load('/data1/lesliec/sarthak/data/chrombpnet_test/hg38_tokenized.npz')
genome_loaded = {}
for key in genome:
    genome_loaded[key] = genome[key][:]
    

In [ ]:
for i in range(10):
    chrom = 'chr'+ str(qtls.iloc[i]['chrom'])
    pos = qtls.iloc[i]['qtl_loc']
    print(f'Chromosome: {chrom}, Position: {pos}')
    print(f'Ref: {qtls.iloc[i]["ref"]}, Alt: {qtls.iloc[i]["alt"]}')
    print(f'true values: {genome_loaded[chrom][pos]}, and {genome_loaded[chrom][pos-1]}')
    print('\n')
#clearly it's the position -1, so it uses 1 indexing for variants

Chromosome: chr1, Position: 1374801
Ref: A, Alt: G
true values: 9, and 7


Chromosome: chr1, Position: 1511945
Ref: G, Alt: A
true values: 8, and 9


Chromosome: chr1, Position: 2548544
Ref: A, Alt: G
true values: 10, and 7


Chromosome: chr1, Position: 2557169
Ref: T, Alt: C
true values: 7, and 10


Chromosome: chr1, Position: 6236178
Ref: T, Alt: G
true values: 9, and 10


Chromosome: chr1, Position: 9235385
Ref: T, Alt: C
true values: 7, and 10


Chromosome: chr1, Position: 28247532
Ref: T, Alt: A
true values: 7, and 10


Chromosome: chr1, Position: 40699407
Ref: C, Alt: A
true values: 10, and 8


Chromosome: chr1, Position: 46198577
Ref: A, Alt: G
true values: 9, and 7


Chromosome: chr1, Position: 46340721
Ref: T, Alt: C
true values: 8, and 10




In [ ]:
#now do for promoterai variants
for i in range(100):
    chrom = promoterai.iloc[i]['chrom']
    pos = promoterai.iloc[i]['pos']
    print(f'Chromosome: {chrom}, Position: {pos}')
    print(f'Ref: {promoterai.iloc[i]["ref"]}, Alt: {promoterai.iloc[i]["alt"]}')
    print(f'true values: {genome_loaded[chrom][pos]}, and {genome_loaded[chrom][pos-1]}')
    assert genome_loaded[chrom][pos-1] == mapping_nuc[promoterai.iloc[i]['ref']]
    print('\n')

#definitely true, so same indexing mechanism

Chromosome: chr1, Position: 64918
Ref: T, Alt: A
true values: 7, and 10


Chromosome: chr1, Position: 64918
Ref: T, Alt: C
true values: 7, and 10


Chromosome: chr1, Position: 64918
Ref: T, Alt: G
true values: 7, and 10


Chromosome: chr1, Position: 64919
Ref: A, Alt: C
true values: 7, and 7


Chromosome: chr1, Position: 64919
Ref: A, Alt: G
true values: 7, and 7


Chromosome: chr1, Position: 64919
Ref: A, Alt: T
true values: 7, and 7


Chromosome: chr1, Position: 64920
Ref: A, Alt: C
true values: 7, and 7


Chromosome: chr1, Position: 64920
Ref: A, Alt: G
true values: 7, and 7


Chromosome: chr1, Position: 64920
Ref: A, Alt: T
true values: 7, and 7


Chromosome: chr1, Position: 64921
Ref: A, Alt: C
true values: 7, and 7


Chromosome: chr1, Position: 64921
Ref: A, Alt: G
true values: 7, and 7


Chromosome: chr1, Position: 64921
Ref: A, Alt: T
true values: 7, and 7


Chromosome: chr1, Position: 64922
Ref: A, Alt: C
true values: 8, and 7


Chromosome: chr1, Position: 64922
Ref: A, Alt: G

In [20]:
len(qtls['chrom'].unique()), qtls['chrom'].unique() #so can't just exclude chromosomes...

(23,
 array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23]))

In [11]:
#print dtypes of each row
for col in promoterai.columns:
    print(col, promoterai[col].dtype)

chrom object
pos int64
ref object
alt object
gene object
gene_id object
transcript_id object
strand int64
tss_pos int64
promoterAI float64


In [18]:
matches = []
with open('/data1/lesliec/sarthak/data/promoterAI/promoterAI_tss500.tsv', 'r') as f:
    for i,line in enumerate(f):
        # print(line)
        print(line.split('\t'))

        if i == 2:
            break

['chrom', 'pos', 'ref', 'alt', 'gene', 'gene_id', 'transcript_id', 'strand', 'tss_pos', 'promoterAI\n']
['chr1', '64918', 'T', 'A', 'OR4F5', 'ENSG00000186092', 'ENST00000641515.2', '1', '65418', '-0.0024\n']
['chr1', '64918', 'T', 'C', 'OR4F5', 'ENSG00000186092', 'ENST00000641515.2', '1', '65418', '-0.0076\n']


In [64]:
from tqdm import tqdm
matches = {}
idx = 0
current_qtl = qtls.iloc[idx]
with open('/data1/lesliec/sarthak/data/promoterAI/promoterAI_tss500.tsv', 'r') as f:
    for i,line in tqdm(enumerate(f), total=261666406):
        if i == 0:
            continue
        # print(line)
        splitline = line.split('\t')
        try:
            chrom = int(splitline[0][3:])
        except ValueError:
            chrom = 23 # for chrX
        pos = int(splitline[1])
        
        while chrom > current_qtl['chrom']: #if there's a jump in chromosomes
            matches[idx] = None
            idx += 1
            current_qtl = qtls.iloc[idx]
            print(f'No match at {chrom}:{pos}, moving to next qtl, distance: {current_qtl["distance"]}')

        if chrom == current_qtl['chrom'] and pos == current_qtl['qtl_loc']:
            matches[idx] = splitline
            idx += 1
            current_qtl = qtls.iloc[idx]
            print(f'Found match at {chrom}:{pos}')
        
        while chrom == current_qtl['chrom'] and pos > current_qtl['qtl_loc']: #do a while loop because maybe next one is also just behind, means we could miss some
            matches[idx] = None
            idx += 1
            current_qtl = qtls.iloc[idx]
            print(f'No match at {chrom}:{pos}, moving to next qtl, distance: {current_qtl["distance"]}')

#most of the ones it skips are over 5k away, so it makes sense?
#also do we really ahve 4 eqtls stacke don one another?
#oh, we need a while loop and it's because jumps to next TSS!!!

  0%|          | 330606/261666406 [00:02<33:02, 131852.75it/s]

Found match at 1:1374801


  0%|          | 410490/261666406 [00:03<32:50, 132609.99it/s]

Found match at 1:1511945


  0%|          | 755423/261666406 [00:05<33:13, 130907.02it/s]

...

Found match at 22:46762547


 96%|█████████▌| 251567392/261666406 [31:07<01:27, 115698.09it/s]

No match at 23:2500054, moving to next qtl, distance: -415


 97%|█████████▋| 252858607/261666406 [31:18<01:05, 134614.81it/s]


IndexError: single positional indexer is out-of-bounds

In [66]:
len(matches)

342

In [ ]:
#lots of ones that don't match, it's a bit strange, but we'll check later?
#let's see how many don't have matches, if distance is greater than 5k, might not match
nonetotal = 0
for key in matches:
    if matches[key] is None:
        nonetotal += 1
print(f'Total non-matches: {nonetotal}') #that's almost half lmfao! Ok we can subset qtls to only ones where it has a prediction, because it's so short range?

Total non-matches: 159


In [68]:
#let's save this information out
#first have to get idx information from the qtls

qtls.loc[0]

label                 0
qtl_idx               0
gene_idx          10057
chrom                14
gene_start     23953734
gene_end       23969279
strand                1
qtl_loc        23953727
ref                   C
alt                   G
sign_target          -1
distance              7
Name: 0, dtype: object

In [69]:
#let's get the indices out of qtls
qtls.index

Index([1138, 1169,  554,    3, 1188,  895,  744, 1077, 1161,  689,
       ...
       1266, 1313,  179, 1101, 1153, 1173,  864, 1248, 1286, 1207],
      dtype='int64', length=342)

In [75]:
matches[10]

['chr1',
 '46340757',
 'A',
 'C',
 'NSUN4',
 'ENSG00000117481',
 'ENST00000474844.6',
 '1',
 '46340806',
 '-0.1116\n']

In [ ]:
indices = qtls.index

preds = np.zeros(1331, dtype='float64')

for i,key in enumerate(matches):
    if matches[key] is not None:
        preds[indices[i]] = float(matches[key][-1][:-1]) #\n is one character

In [83]:
preds

array([-0.3109, -0.129 ,  0.    , ...,  0.    ,  0.    ,  0.    ])

In [84]:
indices[10]

690

In [85]:
preds[690]

-0.1116

In [86]:
#now we can save this out
np.save('/data1/lesliec/sarthak/data/joint_playground/eQTL/EPCOTv2_LCLs/promoterAI_predictions.npy', preds)